간단한 자동사냥 RPG 게임을 만들어봅시다!
사용자의 이름을 입력 받아 플레이어를 생성하고,
몬스터들을 차례로 자동사냥하는 게임을 만들어보고자 합니다.

- **Character 클래스**
    - 이름, 레벨, 체력, 공격력, 방어력의 속성을 가지고 있습니다.
    - 인스턴스의 현재 체력이 0 이상인지를 불리언 값으로 반환하는 `is_alive` 메서드 만들기
    - 공격을 받았을 때, (받은 데미지 - 본인의 방어력)만큼 현재 체력이 감소하는 `take_damage` 메서드 만들기
        - 본인의 방어력이 데미지보다 크다면 체력 감소하지 않음
    - 타겟에게 데미지를 입히는 `attack_target` 메서드 만들기
        - 데미지는 1부터 공격력 사이의 랜덤한 정수

In [122]:
class Character:
    def __init__(self, name, level, hp, attack, defense):
        self.name = name  #이름
        self.level = level  #레벨
        self.hp = hp  #체력
        self.attack = attack  #공격력
        self.defense = defense  #방어력

    def is_alive(self):
        return self.hp > 0

    def take_damage(self, damage):
        actual_damage = 0
        if damage >self.defense:
            actual_damage = damage - self.defense

        self.hp -= actual_damage
        return actual_damage

    def attack_target(self, target):
        damage = random.randint(1, self.attack)
        actual_damage = target.take_damage(damage)
        return actual_damage

- **Player 클래스**
    - Player 클래스는 Character 클래스를 상속 받습니다.
    - 인스턴스 생성 시 레벨 1, 체력 100, 공격력 25, 방어력 5로 초기화합니다.
    - Player 클래스는 경험치 속성을 추가로 가지고 있습니다.
    - 인수로 받은 정수 만큼 경험치를 획득하는 `gain_experience` 메서드 만들기
    - 현재 경험치가 50이상이면 레벨을 1, 공격력을 10, 방어력을 5씩 올리는 `level_up` 메서드 만들기

In [123]:
class Player(Character):  # Character 클래스를 상속받아 Player 클래스 정의
    def __init__(self, name):
        # 부모 클래스 (Character)의 생성자 호출, 레벨 1, 체력 100, 공격력 25, 방어력 5로 초기화
        super().__init__(name, level=1, hp=100, attack=25, defense=5)
        self.experience = 0  # 경험치 속성 추가

    def gain_experience(self, exp):
        self.experience += exp  # 경험치 획득

    def level_up(self):
        if self.experience >= 50:  # 경험치가 50 이상일 동안 반복
            self.level += 1
            self.attack += 10
            self.defense += 5
            self.experience -= 50  # 레벨업에 사용된 경험치 차감

- **Monster 클래스**
    - Monster 클래스는 Character 클래스를 상속 받습니다.
    - 몬스터 생성 시 레벨에 비례하는 체력, 공격력, 방어력을 초기화합니다.
        - 체력: 10~30 사이의 랜덤한 정수 * 레벨
        - 공격력: 5~20 사이의 랜덤한 정수 * 레벨
        - 방어력: 1~5 사이의 랜덤한 정수 * 레벨

In [124]:
class Monster(Character):
    def __init__(self, name, level):
        self.name = name  # 몬스터의 이름을 저장하기 위해 이 줄을 추가합니다.
        self.level = level  # 레벨을 저장하기 위해 이 줄을 추가합니다.
        self.hp = random.randint(10, 30) * level
        self.attack = random.randint(5, 20) * level
        self.defense = random.randint(1, 5) * level

- **Battle 함수**
    - Player 인스턴스와 Monster 인스턴스를 인수로 받아 둘 중 하나의 체력이 0 이하가 될 때까지 공격을 주고 받는 함수
    - 만약 Player 인스턴스가 살아남았다면
        - Player 인스턴스에 (몬스터 레벨 * 20)만큼의 경험치를 추가
        - 경험치가 50 이상이 되었다면 Player의 레벨업 메서드 호출
        - ‘전투 승리!’를 출력
    - Player 인스턴스가 살아남지 못했을 경우
        - ‘전투 패배..’를 출력하기

In [125]:
class Battle:
    def __init__(self, player, monster):
        self.player = player
        self.monster = monster

        while self.player.is_alive() and self.monster.is_alive():
            # 플레이어 공격
            player_damage = self.player.attack_target(self.monster)  # 데미지 값 받아오기
            print(f"{self.player.name}이(가) {self.monster.name}에게 {player_damage}의 데미지를 입혔습니다! {self.monster.name}의 HP: {self.monster.hp}") # 남은 HP 출력
            if self.monster.is_alive():
                # 몬스터 공격
                monster_damage = self.monster.attack_target(self.player)
                print(f"{self.monster.name}이(가) {self.player.name}에게 {monster_damage}의 데미지를 입혔습니다! {self.player.name}의 HP: {self.player.hp}")
                print('')

        if self.player.is_alive():
            self.player.gain_experience(monster.level * 20)
            print('\n전투 승리!')
            print(f'{player.experience}경험치 획득')
            if self.player.experience >= 50:
                self.player.level_up()
                print(f'레벨업! 남은 경험치 :{player.experience}')
            print('-'* 100)
        else:
            print('전투 패배..')


- **Main 함수**
    - 몬스터의 이름, 레벨이 매핑된 딕셔너리 정의하기
        - monster_dict = {‘슬라임’: 1, ‘고블린’: 2, ‘오크’: 3}
    - 사용자로부터 이름을 입력받아 Player 인스턴스 생성하기
    - 몬스터 딕셔너리로부터 Monster 인스턴스 생성하기
    - 생성된 Monster 인스턴스와 Player 인스턴스가 battle 함수를 통해 전투
        - Player는 생성된 몬스터 3마리(슬라임, 고블린, 오크)와 모두 전투해야함
    - 전투 도중에 Player가 사망하면 이후 전투를 진행하지 않고 ‘게임오버’ 출력하고 종료

In [126]:
import random


def main():
    monster_dict = {'슬라임': 1, '고블린': 2, '오크': 3}
    player_name = input("플레이어 이름을 입력하세요: ")
    player = Player(player_name)

    for monster_name, monster_level in monster_dict.items():
        monster = Monster(monster_name, monster_level)
        print(f"\n{monster_name} 등장!")
        print(f'''{player.name} 상태:
        hp : {player.hp}
        공격력: {player.attack}
        방어력: {player.defense}''')
        print(f'''{monster_name} 상태:
        hp : {monster.hp}
        공격력: {monster.attack}
        방어력: {monster.defense}''')
        battle = Battle(player, monster)
        if not player.is_alive():
            print("게임오버")
            break

if __name__ == "__main__":
    main()


플레이어 이름을 입력하세요: 나래

슬라임 등장!
나래 상태:
        hp : 100
        공격력: 25
        방어력: 5
슬라임 상태:
        hp : 13
        공격력: 12
        방어력: 2
나래이(가) 슬라임에게 2의 데미지를 입혔습니다! 슬라임의 HP: 11
슬라임이(가) 나래에게 0의 데미지를 입혔습니다! 나래의 HP: 100

나래이(가) 슬라임에게 13의 데미지를 입혔습니다! 슬라임의 HP: -2

전투 승리!
20경험치 획득
----------------------------------------------------------------------------------------------------

고블린 등장!
나래 상태:
        hp : 100
        공격력: 25
        방어력: 5
고블린 상태:
        hp : 50
        공격력: 36
        방어력: 8
나래이(가) 고블린에게 0의 데미지를 입혔습니다! 고블린의 HP: 50
고블린이(가) 나래에게 27의 데미지를 입혔습니다! 나래의 HP: 73

나래이(가) 고블린에게 13의 데미지를 입혔습니다! 고블린의 HP: 37
고블린이(가) 나래에게 0의 데미지를 입혔습니다! 나래의 HP: 73

나래이(가) 고블린에게 9의 데미지를 입혔습니다! 고블린의 HP: 28
고블린이(가) 나래에게 30의 데미지를 입혔습니다! 나래의 HP: 43

나래이(가) 고블린에게 9의 데미지를 입혔습니다! 고블린의 HP: 19
고블린이(가) 나래에게 27의 데미지를 입혔습니다! 나래의 HP: 16

나래이(가) 고블린에게 4의 데미지를 입혔습니다! 고블린의 HP: 15
고블린이(가) 나래에게 6의 데미지를 입혔습니다! 나래의 HP: 10

나래이(가) 고블린에게 13의 데미지를 입혔습니다! 고블린의 HP: 2
고블린이(가) 나래에게 23의 데미지를 입혔습니다! 나래의 HP: -13

전투 패배..
게임오버


회고

1. 파이썬을 전체적으로 복습할 수 있어서 좋았다
2. 만들고 싶은대로 뚝딱 만들어 지지 않아서 아쉬웠다.
3.데미지에서 방어력을 빼고 계산을 하고 싶은데 그게 안되서 슬프다.